In [ ]:
[ord(x) for x in "hello ⑥"]

In [3]:
text="😀！Unicode shfjvjavja  阿凡达噶看过v卡怪v看 ahghfakhvl ehfjagukjfgjagfjkgfkjgrujkfawwagf"
tokens=text.encode("utf-8")
print(tokens)
print("length:",len(tokens))
tokens=list(map(int,tokens))
print('---')
print(text)
print("length:",len(text))
print('---')
print(tokens)
print("length:",len(tokens))

b'\xf0\x9f\x98\x80\xef\xbc\x81Unicode shfjvjavja  \xe9\x98\xbf\xe5\x87\xa1\xe8\xbe\xbe\xe5\x99\xb6\xe7\x9c\x8b\xe8\xbf\x87v\xe5\x8d\xa1\xe6\x80\xaav\xe7\x9c\x8b ahghfakhvl ehfjagukjfgjagfjkgfkjgrujkfawwagf'
length: 101
---
😀！Unicode shfjvjavja  阿凡达噶看过v卡怪v看 ahghfakhvl ehfjagukjfgjagfjkgfkjgrujkfawwagf
length: 78
---
[240, 159, 152, 128, 239, 188, 129, 85, 110, 105, 99, 111, 100, 101, 32, 115, 104, 102, 106, 118, 106, 97, 118, 106, 97, 32, 32, 233, 152, 191, 229, 135, 161, 232, 190, 190, 229, 153, 182, 231, 156, 139, 232, 191, 135, 118, 229, 141, 161, 230, 128, 170, 118, 231, 156, 139, 32, 97, 104, 103, 104, 102, 97, 107, 104, 118, 108, 32, 101, 104, 102, 106, 97, 103, 117, 107, 106, 102, 103, 106, 97, 103, 102, 106, 107, 103, 102, 107, 106, 103, 114, 117, 106, 107, 102, 97, 119, 119, 97, 103, 102]
length: 101


In [ ]:
def get_stats(ids):
    counts={}
    for pair in zip(ids,ids[1:]):
        counts[pair]=counts.get(pair,0)+1
    return counts

stats=get_stats(tokens)
print(sorted(((v,k) for k,v in stats.items()),reverse=True))
print(stats)

In [ ]:
top_pair=max(stats,key=stats.get)
top_pair

In [ ]:
def merge(ids,pair,idx):
    newids = []
    i=0
    while i <len(ids):
        if i < len(ids)-1 and ids[i] == pair[0] and ids[i+1]==pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids
print(merge([5,6,6,7,9,1],(6,7),99))
tokens2=merge(tokens,top_pair,256)
print(tokens2)
print("length:",len(tokens2))


In [ ]:
vocab_size=276
num_merges=vocab_size-256
ids=list(tokens)
merges={}
for i in range(num_merges):
    stats=get_stats(ids)
    pair=max(stats,key=stats.get)
    idx=256+i
    print(f"merging {pair} into  a new token {idx}")
    ids=merge(ids,pair,idx)
    merges[pair]=idx
print("length:",len(ids))
print(f"compression ratio:{len(tokens)/len(ids):.2f}X")

In [ ]:
vocab={idx:bytes([idx]) for idx in range(256)}
for (p0,p1),idx in merges.items():
    vocab[idx]=vocab[p0]+vocab[p1]  #连接

def decode(ids):
    tokens=b"".join(vocab[idx] for idx in ids)
    text=tokens.decode("utf-8",errors="replace")
    return text

print(decode([128]))

In [ ]:
def encode(text):
    tokens=list(text.encode("utf-8"))
    while len(tokens)>=2:
        stats=get_stats(tokens)
        pair=min(stats,key=lambda p:merges.get(p,float("inf")))  
        #从 tokens取具有最小值的连续对
        if pair not in merges:
            break
        idx=merges[pair]
        tokens=merge(tokens,pair,idx)
    return tokens

print(encode("hello world!dahfjdasvkjva;;"))        

In [ ]:
print(decode(encode("hello world!dahfjdasvkjva;;")))

In [ ]:
text2=decode(encode(text))
print(text2==text)

In [ ]:
import regex as re

In [ ]:
gpt2pat = re.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)
print(re.findall(gpt2pat,"Hello world 123 how are you 'sdajhc"))  #merge是（ 单词）间的合并

In [ ]:
import tiktoken
enc=tiktoken.get_encoding("cl100k_base")
print(enc.decode(enc.encode("fbjsrf jsjej"))=="fbjsrf jsjej")